In [31]:
# Imports and loading data

import pickle
import numpy as np
import pandas as pd
from scipy.stats import linregress
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, RepeatVector, TimeDistributed, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from prophet import Prophet
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

with open('../../data/ch1_etfs_valid_pairs.pkl', 'rb') as f:
    valid_pairs = pickle.load(f)

with open('../../data/ch0_etfs.pkl', 'rb') as f:
    df, df_returns = pickle.load(f)

In [55]:
def optimize_arima(data, p_range, d_range, q_range):
    best_aic = float("inf")
    best_params = None
    for p, d, q in product(p_range, d_range, q_range):
        try:
            model = ARIMA(data, order=(p, d, q))
            results = model.fit()
            aic = results.aic
            if aic < best_aic:
                best_aic = aic
                best_params = (p, d, q)
        except:
            continue
    return best_params

evaluation_results = []

for pair in valid_pairs:

    evaluation_dict = {}
    
    ticker_1 = pair[0]
    ticker_2 = pair[1]

    evaluation_dict["Name"] = f'{ticker_1} and {ticker_2}'

    ticker_series_1 = df[ticker_1]
    ticker_series_2 = df[ticker_2]

    slope, intercept, _, _, _ = linregress(ticker_series_1, ticker_series_2)
    spread = ticker_series_2 - (slope * ticker_series_1 + intercept)

    data = spread.values

    scaler = MinMaxScaler(feature_range=(0, 1))
    data = scaler.fit_transform(data.reshape(-1, 1)).reshape(-1)

    def create_sequences(data, input_length, output_length):
        X, y = [], []
        for i in range(len(data) - input_length - output_length + 1):
            X.append(data[i:i + input_length])
            y.append(data[i + input_length:i + input_length + output_length])
        return np.array(X), np.array(y)

    input_length = 3
    output_length = 1

    train_size = int(len(data) * 0.8)
    train_data = data[:train_size]
    test_data = data[train_size:]

    X_train, y_train = create_sequences(train_data, input_length, output_length)
    X_test, y_test = create_sequences(test_data, input_length, output_length)

    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], 1))

    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
    y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], 1))
    
    inputs = Input(shape=(input_length, 1))
    encoder = LSTM(5, activation='relu', return_sequences=True)(inputs)
    encoder = Dropout(0.2)(encoder)
    encoder = LSTM(5, activation='relu')(encoder)
    encoder = Dropout(0.2)(encoder)
    encoder = RepeatVector(output_length)(encoder)
    
    decoder = LSTM(5, activation='relu', return_sequences=True)(encoder)
    decoder = Dropout(0.2)(decoder)
    decoder = LSTM(5, activation='relu', return_sequences=True)(decoder)
    decoder = Dropout(0.2)(decoder)
    outputs = TimeDistributed(Dense(1))(decoder)
    
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mse')
    model.summary()

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    history = model.fit(X_train, y_train, epochs=5, batch_size=16, validation_split=0.2, callbacks=[early_stopping])


    test_loss = model.evaluate(X_test, y_test)
    evaluation_dict["LSTMED Test Loss"] = test_loss

    plt.figure(figsize=(12, 6))
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('LSTMED Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(f'../../assets/etfs/{ticker_1}_{ticker_2}/lstmed_loss_plot.png')
    plt.close()

    predicted_train = model.predict(X_train)
    predicted_train = scaler.inverse_transform(predicted_train.reshape(-1, 1)).reshape(predicted_train.shape)
    y_train_true = scaler.inverse_transform(y_train.reshape(-1, 1)).reshape(y_train.shape)

    y_train_true_trend = []
    for i in range(1, len(y_train_true)):
        if np.mean(y_train_true[i,:]) > np.mean(y_train_true[i-1,:]):
            y_train_true_trend.append(1)
        elif np.mean(y_train_true[i,:]) < np.mean(y_train_true[i-1,:]):
            y_train_true_trend.append(-1)
        else:
            y_train_true_trend.append(0)

    predicted_train_trend = []
    for i in range(1, len(predicted_train)):
        if np.mean(predicted_train[i,:]) > np.mean(predicted_train[i-1,:]):
            predicted_train_trend.append(1)
        elif np.mean(predicted_train[i,:]) < np.mean(predicted_train[i-1,:]):
            predicted_train_trend.append(-1)
        else:
            predicted_train_trend.append(0)

    predicted_test = model.predict(X_test)
    predicted_test = scaler.inverse_transform(predicted_test.reshape(-1, 1)).reshape(predicted_test.shape)
    y_test_true = scaler.inverse_transform(y_test.reshape(-1, 1)).reshape(y_test.shape)

    y_test_true_trend = []
    for i in range(1, len(y_test_true)):
        if np.mean(y_test_true[i,:]) > np.mean(y_test_true[i-1,:]):
            y_test_true_trend.append(1)
        elif np.mean(y_test_true[i,:]) < np.mean(y_test_true[i-1,:]):
            y_test_true_trend.append(-1)
        else:
            y_test_true_trend.append(0)

    predicted_test_trend = []
    for i in range(1, len(predicted_test)):
        if np.mean(predicted_test[i,:]) > np.mean(predicted_test[i-1,:]):
            predicted_test_trend.append(1)
        elif np.mean(predicted_test[i,:]) < np.mean(predicted_test[i-1,:]):
            predicted_test_trend.append(-1)
        else:
            predicted_test_trend.append(0)

    plt.figure(figsize=(12, 6))
    plt.plot(y_train_true[:,0], label='True')
    plt.plot(predicted_train[:,0], label='Predicted')
    plt.title('LSTMED True vs Predicted Values (Sample from Train Set)')
    plt.legend()
    plt.savefig(f'../../assets/etfs/{ticker_1}_{ticker_2}/lstmed_train_predictions_plot.png')
    plt.close()

    plt.figure(figsize=(12, 6))
    plt.plot(y_test_true[:,0], label='True')
    plt.plot(predicted_test[:,0], label='Predicted')
    plt.title('LSTMED True vs Predicted Values (Sample from Test Set)')
    plt.legend()
    plt.savefig(f'../../assets/etfs/{ticker_1}_{ticker_2}/lstmed_test_predictions_plot.png')
    plt.close()

    accuracy_trend_train = [y_train_true_trend[i] == predicted_train_trend[i] for i in range(len(predicted_train_trend))]
    accuracy_trend_test = [y_test_true_trend[i] == predicted_test_trend[i] for i in range(len(predicted_test_trend))]

    evaluation_dict["LSTMED Train Trend Accuracy"] = np.sum(accuracy_trend_train)/len(accuracy_trend_train)
    evaluation_dict["LSTMED Test Trend Accuracy"] = np.sum(accuracy_trend_test)/len(accuracy_trend_test)

    plt.figure(figsize=(12, 6))
    for i in range(len(y_test_true_trend)):
        if y_test_true_trend[i] == predicted_test_trend[i]:
            plt.scatter(i, y_test_true_trend[i] - predicted_test_trend[i], c='g')
        elif y_test_true_trend[i] > predicted_test_trend[i]:
            plt.scatter(i, y_test_true_trend[i] - predicted_test_trend[i], c='r')
        else:
            plt.scatter(i, y_test_true_trend[i] - predicted_test_trend[i], c='b')
    red_patch = mpatches.Patch(color='red', label='Predicted Downwards Trend, True Upwards Trend')
    green_patch = mpatches.Patch(color='green', label='Correct Prediction')
    blue_patch = mpatches.Patch(color='blue', label='Predicted Upwards Trend, True Downwards Trend')
    plt.ylim(-5,5)
    plt.title('LSTMED True vs Predicted Trend Diff (Sample from Test Set)')
    plt.legend(handles=[red_patch, green_patch, blue_patch])
    plt.savefig(f'../../assets/etfs/{ticker_1}_{ticker_2}/lstmed_test_trend_diff_plot.png')
    plt.close()

    y_test_true_ma_adjusted = [np.mean(y_test_true[i,:]) for i in range(len(y_test_true))]
    predicted_test_ma_adjusted = [np.mean(predicted_test[i,:,:]) for i in range(len(predicted_test))]

    y_train_true_ma_adjusted = [np.mean(y_train_true[i,:]) for i in range(len(y_train_true))]
    predicted_train_ma_adjusted = [np.mean(predicted_train[i,:,:]) for i in range(len(predicted_train))]

    plt.figure(figsize=(12, 6))
    plt.plot(y_test_true_ma_adjusted, label='True MA')
    plt.plot(predicted_test_ma_adjusted, label='Predicted MA')
    plt.title('LSTMED True vs Predicted MAs (Sample from Test Set)')
    plt.legend()
    plt.savefig(f'../../assets/etfs/{ticker_1}_{ticker_2}/lstmed_test_predictions_ma_plot.png')
    plt.close()

    plt.figure(figsize=(12, 6))
    plt.plot(y_train_true_ma_adjusted, label='True MA')
    plt.plot(predicted_train_ma_adjusted, label='Predicted MA')
    plt.title('LSTMED True vs Predicted MAs (Sample from Train Set)')
    plt.legend()
    plt.savefig(f'../../assets/etfs/{ticker_1}_{ticker_2}/lstmed_train_predictions_ma_plot.png')
    plt.close()

    def create_lagged_features(series, lags=5):
        spread = pd.Series(series, name='spread')
        df = pd.DataFrame(spread)
        for lag in range(1, lags + 1):
            df[f'lag_{lag}'] = df['spread'].shift(lag)
        df.dropna(inplace=True)
        return df

    lags = 5
    lagged_df = create_lagged_features(data, lags=lags)

    X = lagged_df.drop(columns=['spread'])
    y = lagged_df['spread']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)
    rf_predictions = rf_model.predict(X_test)

    xgb_model = xgb.XGBRegressor(n_estimators=100, random_state=42)
    xgb_model.fit(X_train, y_train)
    xgb_predictions = xgb_model.predict(X_test)

    rf_mse = mean_squared_error(y_test, rf_predictions)
    rf_mae = mean_absolute_error(y_test, rf_predictions)
    evaluation_dict["Random Forest MSE"] = rf_mse
    evaluation_dict["Random Forest MAE"] = rf_mae

    xgb_mse = mean_squared_error(y_test, xgb_predictions)
    xgb_mae = mean_absolute_error(y_test, xgb_predictions)
    evaluation_dict["XGBoost MSE"] = xgb_mse
    evaluation_dict["XGBoost MAE"] = xgb_mae

    plt.figure(figsize=(15, 6))
    plt.title('Random Forest and XGBoost Predictions')
    plt.plot(y_test.index, y_test, label='True Values', color='blue')
    plt.plot(y_test.index, rf_predictions, label='Random Forest Predictions', color='green')
    plt.plot(y_test.index, xgb_predictions, label='XGBoost Predictions', color='red')
    plt.legend()
    plt.savefig(f'../../assets/etfs/{ticker_1}_{ticker_2}/spread_forecasting_rf_xgb_predictions_plot.png')
    plt.close()

    trend_rf = []
    for i in range(1, len(y_test)-10):
        if rf_predictions[i] > y_test.iloc[i-1]:
            trend_rf.append(1)
        elif rf_predictions[i] < y_test.iloc[i-1]:
            trend_rf.append(-1)
        else:
            trend_rf.append(0)

    trend_rf_true = []
    for i in range(1, len(y_test)-10):
        if np.mean(y_test.iloc[i:i+10]) > y_test.iloc[i-1]:
            trend_rf_true.append(1)
        elif np.mean(y_test.iloc[i:i+10]) < y_test.iloc[i-1]:
            trend_rf_true.append(-1)
        else:
            trend_rf_true.append(0)

    accuracy_trend = [trend_rf[i] == trend_rf_true[i] for i in range(len(trend_rf))] 
    evaluation_dict["Random Forest Trend Accuracy"] = np.sum(accuracy_trend)/len(accuracy_trend)

    trend_xgb = []
    for i in range(1, len(y_test)-10):
        if xgb_predictions[i] > y_test.iloc[i-1]:
            trend_xgb.append(1)
        elif xgb_predictions[i] < y_test.iloc[i-1]:
            trend_xgb.append(-1)
        else:
            trend_xgb.append(0)

    trend_xgb_true = []
    for i in range(1, len(y_test)-10):
        if np.mean(y_test.iloc[i:i+10]) > y_test.iloc[i-1]:
            trend_xgb_true.append(1)
        elif np.mean(y_test.iloc[i:i+10]) < y_test.iloc[i-1]:
            trend_xgb_true.append(-1)
        else:
            trend_xgb_true.append(0)

    accuracy_trend = [trend_xgb[i] == trend_xgb_true[i] for i in range(len(trend_xgb))] 
    evaluation_dict["XGBoost Trend Accuracy"] = np.sum(accuracy_trend)/len(accuracy_trend)

    prophet_df = pd.DataFrame({'ds': pd.to_datetime(ticker_series_1.index), 'y': spread})
    for lag in range(1, 11):
        prophet_df[f'lag_{lag}'] = prophet_df['y'].shift(lag)
    prophet_df.dropna(inplace=True)

    prophet_df['ds'] = prophet_df['ds'].dt.tz_localize(None)

    train_size = int(len(prophet_df) * 0.8)
    train_prophet_df = prophet_df.iloc[:train_size]
    test_prophet_df = prophet_df.iloc[train_size:]
        
    prophet_model = Prophet()
    prophet_model.fit(train_prophet_df)

    forecast = prophet_model.predict(test_prophet_df)

    fig = prophet_model.plot_components(forecast)
    plt.savefig(f'../../assets/etfs/{ticker_1}_{ticker_2}/prophet_components_plot.png')
    plt.close()

    f, ax = plt.subplots(figsize=(15, 5))
    ax.scatter(test_prophet_df['ds'], test_prophet_df['y'], color='r')
    fig = prophet_model.plot(forecast, ax=ax)
    plt.savefig(f'../../assets/etfs/{ticker_1}_{ticker_2}/prophet_predictions_plot.png')
    plt.close()

    
    evaluation_dict["Prophet MSE"] = mean_squared_error(y_true=test_prophet_df['y'],
                    y_pred=forecast['yhat'])
    evaluation_dict["Prophet MAE"] = mean_absolute_error(y_true=test_prophet_df['y'],
                    y_pred=forecast['yhat'])
    
    evaluation_results.append(evaluation_dict)

    arima_data = spread.values

    p_range = range(0, 3)
    d_range = range(0, 2)
    q_range = range(0, 3)
    best_params = optimize_arima(arima_data, p_range, d_range, q_range)

    model = ARIMA(arima_data, order=best_params)
    results = model.fit()

    with open(f'../../data/ch3_{ticker_1}_{ticker_2}_arima.pkl', 'wb') as f:
        pickle.dump(results, f)

    with open(f'../../data/ch3_{ticker_1}_{ticker_2}_random_forest.pkl', 'wb') as f:
        pickle.dump(rf_model, f)

evaluation_results_df = pd.DataFrame(evaluation_results)
evaluation_results_df.to_csv('../../assets/etfs/spread_forecasting_models_evaluation_results.csv', index=False)


Model: "functional_42"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_42 (InputLayer)     │ (None, 3, 1)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_168 (LSTM)                 │ (None, 3, 5)           │           140 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_168 (Dropout)           │ (None, 3, 5)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_169 (LSTM)                 │ (None, 5)              │           220 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_169 (Dropout)           │ (None, 5)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector_42 (RepeatVector) │ (None, 1, 5)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_170 (LSTM)                 │ (None, 1, 5)           │           220 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_170 (Dropout)           │ (None, 1, 5)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_171 (LSTM)                 │ (None, 1, 5)           │           220 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_171 (Dropout)           │ (None, 1, 5)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_42             │ (None, 1, 1)           │             6 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 806 (3.15 KB)

 Trainable params: 806 (3.15 KB)

 Non-trainable params: 0 (0.00 B)

...

 Trainable params: 806 (3.15 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.2294 - val_loss: 0.1597
Epoch 2/5
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1999 - val_loss: 0.1327
Epoch 3/5
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1748 - val_loss: 0.1094
Epoch 4/5
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1501 - val_loss: 0.0892
Epoch 5/5
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1219 - val_loss: 0.0720
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 900us/step - loss: 0.0742
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 856us/step


19:22:22 - cmdstanpy - INFO - Chain [1] start processing
19:22:22 - cmdstanpy - INFO - Chain [1] done processing


Finding optimal ARIMA parameters for USIG and VTC...
Best ARIMA parameters (p, d, q): (1, 0, 1)
